# i.MXRT118x Debug Authentication example
I.MX RT118x offers the Debug Authentication Protocol (DAP) as a mechanism to authenticate the debugger (an external entity) which has the credentials approved by the product manufacturer before granting the debug access to the device. This example demonstrates process how to establish Debug Authentication protocol.

The process could be split into following steps:

## Introduction
The fundamental principles of debugging, which require access to the system state and system information, conflict with the principles of security, which require the restriction of access to assets. Thus, many products disable debug access completely before deploying the product. To address these challenges, the chip offers a debug authentication protocol as a mechanism to authenticate the debugger (an external entity) has the credentials approved by the product manufacturer before granting debug access to the device.
The debug authentication is a challenge-response scheme and assures that only the debugger in possession of the required debug credentials can successfully authenticate over the debug interface and access restricted parts of the device. 

The protocol is divided into steps as described below:
1. The debugger initiates the Debug Mailbox message exchange by setting the CSW[RESYNCH_REQ] bit and
CSW[CHIP_RESET_REQ] bit of DM-AP.
2. The debugger waits (minimum 30 ms) for the devices to restart and enter debug mailbox request handling loop.
3. The debugger sends Debug Authentication Start command (command code 10h) to the device.
4. The device responds back with Debug Authentication Challenge (DAC) packet based on the debug access rights preconfigured in CMPA fields, which are collectively referred as Device Credential Constraints Configuration (DCFG_CC).
The response packet also contains a 32 bytes random challenge vector.
5. The debugger responds to the challenge with a Debug Authentication Response (DAR) message by using an
appropriate debug certificate, matching the device identifier in the DAC. The DAR packet contains the debug access
permission certificate, also referred as Debug Credential (DC), and a cryptographic signature binding the DC and the
challenge vector provided in the DAC.
6. The device on receiving the DAR, validates the contents by verifying the cryptographic signature of the message using
the debugger's public key present in the embedded the Debug Credential (DC). On successful validation of DAR, the
device enables access to the debug domains permitted in the DC

![debug_authentication_flow](../img/debug_authentication_flow.png)

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)
- The device must be properly configured to run signed application (example of run signed application is here: [AHAB Secure boot](../../ahab/rt118x_secure_boot/rt118x_secure_boot.ipynb))
- The device life cycle must be moved to state OEM_CLOSED (see more detail also in [AHAB Secure boot](../../ahab/rt118x_secure_boot/rt118x_secure_boot.ipynb))
- This example uses EVK-MIMXRT1189 board. This is example board configuration with external debugger. In our case we used Segger J-Link probe connected over sandard 20 pin JTAG connector.

  <img src="../../_data/img/boards/rt1189-evk.png" alt="rt1189-evk" height="400">

### 1.1 Let's prepare the environment

In [ ]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [ ]:
WORKSPACE = "workspace/"  # change this to path to your workspace
INPUTS = "inputs/"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

DAT_CONFIG = INPUTS + "dat_config.yaml"  # yaml file with configuration used in this example
DC_CONFIG = (
    INPUTS + "dc_config.yaml"
)  # yaml file with configuration used in this example for debug credential file

# DEBUGGER_PROBE = "pyocd" #onboard debugger on EVK-RT118x rev C
DEBUGGER_PROBE = "pyocd"
# Only one probe can be connected to the board.

FAMILY = "mimxrt1189"

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## 2. Generate RoT + Debug auth keys

First we need to generate Root of Trust Keys (RoTKs)/Super Root Keys (SRKs) and Debug Credential Key (DCK). Use *nxpcrypto* app to generate secp256r1 keys (see [How-to-get-keys-using-nxpcrypto](../../crypto/keys.ipynb)).
We will need by default create 4 RoTKs and 1 DCK.

The notebook is using pre-generated keys just for that example purposes from folder `_data/keys/ecc256` in root of all SPSDK examples.

## 3. Generate debug credential file including its configuration file

First we need to prepare the configuration file for `debug credential file` export. Let's begin by creating a template configuration file using the `nxpdebugmbox dat dc get-template` command. To simplify this example, we have already prepared that configuration, which can be found in the [./inputs/dc_config.yaml](./inputs/dc_config.yaml) file. Below, we'll compare the differences between the template and our customized example to highlight the additions we've made.

Note: As is mentioned in introduction, the DC file should be created by owner of RoT credentials.

Note: Example how to get UUID for one device: Connect debug probe into the connector and read the UUID by `nxpdebugmbox -f mimxrt1189 tool get-uuid`

In [2]:
# Get difference of template and user YAML configuration
YamlDiffWidget("inputs/dc_config.diffc").html

nxpdebugmbox -f mimxrt1189 dat dc get-template -o workspace/dc_config.yaml --force 
The Debug Credentials template for mimxrt1189 has been saved into workspace\dc_config.yaml YAML file


Now we generate dc file based on yaml configuration.

In [13]:
DC_FILE_PATH = WORKSPACE + "debug_auth.dc"
%! nxpdebugmbox -f $FAMILY dat dc export -c $DC_CONFIG -o $DC_FILE_PATH --force

nxpdebugmbox -f mimxrt1189 dat dc export -c inputs/dc_config.yaml -o workspace/debug_auth.dc --force 
RKTH: cb2cc774b2dcec92c840eca0646b78f8d3661d3a43ed265a490a13aca75e190a
Creating Debug credential file succeeded


## 4. Generate debug authentication configuration file

In advance we will prepare the configuration file for debug authentication procedure itself. Let's begin again by creating a template configuration file using the `nxpdebugmbox dat get-template` command. To simplify this example, we have already prepared that configuration, which can be found in the [./inputs/dat_config.yaml](./inputs/dat_config.yaml) file. Below, we'll compare the differences between the template and our customized example to highlight the additions we've made. This tiny configuration file is needed that specify the data from in-field technician.

In [4]:
# Get difference of template and user YAML configuration
YamlDiffWidget("inputs/dat_config.diffc").html

nxpdebugmbox -f mimxrt1189 dat get-template -o workspace/dat_config.yaml --force 
Creating workspace\dat_config.yaml template file.


## 5. Test debug authentication
Now we can call authentication command for nxpdebugmbox. Since the board is in the OEM closed life cycle, as a validation test we will use test-connection command from `nxpdebugmbox` application. After the auth command is performed, the MCU should has enable debug interface by settings in CC_SOCU field in debug credential file. 

In [21]:
# Test to confirm that debug access doesn't work
%! nxpdebugmbox -f $FAMILY -i $DEBUGGER_PROBE mem-tool test-connection

nxpdebugmbox -f mimxrt1189 -i pyocd mem-tool test-connection 
  #   Interface   Id          Description                 
----------------------------------------------------------
  0   PyOCD       851002903   Segger J-Link Compact PLUS  
The device is not-accessible for debugging.


In [22]:
# Unlock the Debug Authentication
%! nxpdebugmbox -f $FAMILY -i $DEBUGGER_PROBE -v dat auth -c $DAT_CONFIG

nxpdebugmbox -f mimxrt1189 -i pyocd -v dat auth -c inputs/dat_config.yaml 
INFO:root:
INFO:root:
INFO:root:--- SPSDK DEBUG LOGGING STARTED (2024-11-06 09:33:11.634502) ---
INFO:root:SPSDK version: 2.3.0.dev196+g291e35f98
INFO:root:Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
INFO:root:
INFO:spsdk.apps.nxpdebugmbox:Starting Debug Authentication
  #   Interface   Id          Description                 
----------------------------------------------------------
  0   PyOCD       851002903   Segger J-Link Compact PLUS  
INFO:spsdk.debuggers.debug_probe_pyocd:PyOCD connected via J-Link Compact PLUS probe.
INFO:spsdk.apps.nxpdebugmbox:DAC: 
Version                : Version 2.0
SOCC                   : 0x5254049C
UUID                   : C8BDB7562AFA425B9528F5D63EB4125E
CC_VU                  : 0
ROTID_rkh_revocation   : 00000000
ROTID_rkth_hash        : cb2cc774b2dcec92c840eca0646b78f8d3661d3a43ed265a490a13aca75e190a
CC_soc_pinned         

In [23]:
# Test to confirm that debug access works
%! nxpdebugmbox -f $FAMILY -i $DEBUGGER_PROBE mem-tool test-connection

nxpdebugmbox -f mimxrt1189 -i pyocd mem-tool test-connection 
  #   Interface   Id          Description                 
----------------------------------------------------------
  0   PyOCD       851002903   Segger J-Link Compact PLUS  
The device is accessible for debugging.
